# Problem 3

# Import Necessary Packages 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


# Obtain and Clean Wikipedia Data from Downloaded csv

In [2]:
df = pd.read_csv('TorontoData.csv')

In [3]:
#Remove Not assigned Borough
df=df[df['Borough']!='Not assigned']

In [4]:
#Check for and join any duplicate Borough's
temp_df=df.groupby('Postal Code')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

#Check for any Not assigned Neighboorhoods
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned',df['Borough'], df['Neighborhood'])

In [5]:
# Reset Index after dropping black Borough's
df.reset_index()
df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
#Find shape of data
df.shape

(103, 3)

# Get Lat/Long Data for DataFrame

In [7]:
# Import Data Provided
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [8]:
#Rename columns to match and merge 
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_combined = pd.merge(geo_df, df, on='Postalcode')

In [9]:
#Rearrange and show first few lines of data
geo_combined=geo_combined[['Postalcode','Borough','Neighborhood', 'Latitude', 'Longitude']]

# Finding Venues in these Neighbourhoods

In [31]:
#Map of Toronto neighboorhood
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=11)

for lat,lng,borough,neighbourhood in zip(geo_combined['Latitude'],geo_combined['Longitude'],geo_combined['Borough'],geo_combined['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

In [11]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'NIVZPXB53TQW5F4IK53IZBTO3HMJPKZI25QCK0Z2PDW1DSCC' # your Foursquare ID
CLIENT_SECRET = 'WL2X4XG1R3CIHKPKVRR2XRM134TMVYHNHUAOPS5OKEORR44K' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NIVZPXB53TQW5F4IK53IZBTO3HMJPKZI25QCK0Z2PDW1DSCC
CLIENT_SECRET:WL2X4XG1R3CIHKPKVRR2XRM134TMVYHNHUAOPS5OKEORR44K


In [12]:
k=15
toronto_clustering = geo_combined.drop(['Postalcode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
geo_combined.insert(0, 'City Cluster', kmeans.labels_)

In [28]:
#Mapping clusters
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(geo_combined['Latitude'], geo_combined['Longitude'], geo_combined['Neighborhood'], geo_combined['City Cluster']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [14]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City Cluster', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
location_venues = getNearbyVenues(names=geo_combined['City Cluster'],
                                   latitudes=geo_combined['Latitude'],
                                   longitudes=geo_combined['Longitude']
                                  )

8
8
8
8
8
2
2
2
2
2
5
5
5
5
5
9
8
9
9
13
13
13
13
13
13
9
9
10
13
4
4
4
4
4
10
10
10
10
0
10
10
10
10
3
0
0
0
0
0
0
3
3
3
3
3
3
3
3
3
3
3
3
13
0
0
7
7
7
3
3
3
6
6
0
6
7
7
7
7
4
6
6
6
7
6
3
1
10
12
12
12
12
12
11
1
1
14
11
11
11
11
14
14


In [16]:
location_venues.head()

,City Cluster,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,8,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,8,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,8,43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,8,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,8,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [17]:
location_venues.groupby('City Cluster').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City Cluster,,,,,,
0,125,125,125,125,125,125
1,20,20,20,20,20,20
2,24,24,24,24,24,24
3,1136,1136,1136,1136,1136,1136
4,26,26,26,26,26,26
5,31,31,31,31,31,31
6,85,85,85,85,85,85
7,225,225,225,225,225,225
8,22,22,22,22,22,22


In [18]:
toronto_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")
#toronto_onehot.drop(['City Cluster'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='City Cluster', value=location_venues['City Cluster'] )
toronto_onehot.shape

(2097, 269)

In [19]:
toronto_grouped = toronto_onehot.groupby('City Cluster').mean().reset_index()
toronto_grouped

,City Cluster,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.008000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.01

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City Cluster']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['City Cluster'] = toronto_grouped['City Cluster']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,City Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Coffee Shop,Park,Sushi Restaurant,Dessert Shop,Sporting Goods Shop,Sandwich Place,Gym,Restaurant,Café,Clothing Store
1,1,Coffee Shop,Hotel,Pizza Place,Café,Fried Chicken Joint,Mediterranean Restaurant,Beer Store,Burrito Place,Sandwich Place,Intersection
2,2,Bakery,Bus Line,Bus Station,Playground,Skating Rink,Soccer Field,Spa,Discount Store,Intersection,Café
3,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Gym,Clothing Store,American Restaurant
4,4,Park,Grocery Store,Korean Restaurant,Bank,Bar,Baseball Field,Basketball Court,Food Truck,Snack Place,Shopping Mall
5,5,Chinese Restaurant,Pharmacy,Indian Restaurant,Pizza Place,Breakfast Spot,Vietnamese Restaurant,Pet Store,Latin American Restaurant,Lounge,Smoke Shop
6,6,Coffee Shop,Café,Pizza Place,Park,Arts & Crafts Store,Pub,Italian Restaurant,Clothing Store,Grocery Store,Sandwich Place
7,7,Café,Bar,Coffee Shop,Restaurant,Grocery Store,Bakery,Italian Restaurant,Park,Breakfast Spot,Vegetarian / Vegan Restaurant
8,8,Coffee Shop,Bank,Hakka Restaurant,Lounge,Bakery,Bar,Intersection,Fast Food Restaurant,Breakfast Spot,Electronics Store
9,9,Clothing Store,Fast Food Restaurant,Coffee Shop,Japanese Restaurant,Bank,Sandwich Place,Chinese Restaurant,Restaurant,Baseball Field,Bakery


In [27]:
score =['1','1','3','0','4','1','2','2','0','1','1','2','1','1','4']

## Summary of Score

| City Cluster | Score  |
| ------------ | ------ |
| 0            | 1      |
| 1            | 1      |
| 2            | 3      | 
| 3            | 0      |
| 4            | 4      | 
| 5            | 1      | 
| 6            | 2      |
| 7            | 2      |
| 8            | 0      |
| 9            | 1      |
| 10           | 1      |
| 11           | 2      |
| 12           | 1      |
| 13           | 1      |
| 14           | 4      |